# Introducción a Spacy
---

En este taller guiado presentaremos una introducción práctica a la librería `spacy` para procesamiento de lenguaje natural. Comenzamos importándola:

In [ ]:
import spacy
import pandas as pd

## **1. ¿Qué es Spacy?**
---

Se trata de una librería para procesamiento de lenguaje natural (NLP) que provee una forma de uso simple y flexible para implementar soluciones de NLP de forma inmediata en aplicaciones industriales.

<img src="https://drive.google.com/uc?export=view&id=1_hGDcMnNp0MAS7ERGPcmnGTX-xAs1iiG" width="40%">

`spacy` tiene las siguientes características:

* Soporta más de 66 lenguajes.
* Trae más de 80 _pipelines_ pre-entrenados para más de 24 lenguajes.
* Permite usar modelos del estado del arte en NLP.
* Embeddings pre-entrenados.
* Velocidad competitiva con el estado del arte.
* Sistema de entrenamiento de modelos.
* Herramientas de tokenizado lingüístico.
* Componentes de distintas tareas generales de NLP como: reconocimiento de entidades nombradas, part-of-speech, segmentación de textos, entre otros.
* Componentes personalizados.
* Soporta modelos personalizados de diversas librerías de machine learning.
* Herramientas de visualización para texto.
* Fácil empaquetado de modelos para el despliegue y la gestión.
* Modelos con un buen desempeño base y con una rigurosa evaluación.

## **2. Modelos de Spacy**
---

`spacy` tiene distintos modelos pre-entrenados y listos para usar en distintos lenguajes. Puedes revisar los distintos lenguajes y modelos, como se muestra a continuación, a partir de [este enlace](https://spacy.io/models):

In [ ]:
#@markdown ##**Ejecute esta celda para ver el video.**
from IPython.display import IFrame
IFrame(
        src="https://drive.google.com/file/d/1ncW1HPrkU7aHw2eUhBl-zeoaukypmIZ0/preview",
        width="768px",
        height="432px"
        )

Los modelos pre-entrenados generalmente tienen un código de nombramiento como `{lang}_{from}_{corpus}_{size}`:

* `lang`: código del idioma de los modelos a cargar.
* `from`: repositorio de dónde vienen los modelos.
* `corpus`: conjunto de datos donde fueron entrenados los modelos.
* `size`: tamaño del modelo, generalmente se maneja `sm` (small - pequeño), `md` (medium - mediano), `lg` (large - grande).

Los modelos los podemos descargar automáticamente usando la función `download` dentro del `cli` de `spacy` como mostramos a continuación:

In [ ]:
spacy.cli.download("es_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## **_3. Pipeline_**
---

Los _pipelines_ de `spacy` nos proveen una interfaz por medio de la cual podemos aplicar distintas técnicas y modelos de NLP de forma ordenada y secuencial. Un _pipeline_ representa una serie de pasos (componentes) que se aplican uno tras otro para obtener información específica de un texto. Por lo general siguen la siguiente estructura:

<center>
<img src="https://drive.google.com/uc?export=view&id=12MczaMlyMo12kIbk5FZC0PzA1h5E6fQq" width="60%">
<font size="1" color="black"><i>CNN/CPU pipeline design [Imagen]. Extraída de https://spacy.io/models </i></font>
</center>


Cada uno de los componentes del _pipeline_ será explicado en detalle posteriormente en este mismo taller guiado. Por ahora, podemos centrarnos en el proceso, el cual consiste en los siguientes pasos:

* Los componentes `tagger`, `morphologizer` y `parser` esperan la salida del componente `tok2vec`. Si el `lemmatizer` es entrenable, este también espera la salida de `tok2vec`.
* El componente `attribute_ruler` utiliza los resultados del `tagger` y valida que los espacios y caracteres especiales estén etiquetados correctamente.
* El `lemmatizer` utiliza las reglas obtenidas del `attribute_ruler` para transformar los textos.
* El componente `ner` es independiente del resto de etapas y puede tener su propio componente de `tok2vec`.

Veamos cómo crear un _pipeline_ de `spacy` con la función `load` y el _pipeline_ pre-entrenado que descargamos anteriormente:

In [ ]:
nlp = spacy.load("es_core_news_sm")
print(nlp)

Como puede se puede ver, cargamos un objeto de tipo `Spanish`, lo que indica que `spacy` por detrás define clases personalizadas para los distintos lenguajes que queramos manejar.

Podemos obtener el listado de los componentes que tiene el _pipeline_ que cargó usando el atributo `component_names`, como se muestra a continuación:

In [ ]:
print(nlp.component_names)

['tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner']


Generalmente, un _pipeline_ de `spacy` aplica todos los componentes que tiene definidos sobre un documento específico, no obstante, en algunas aplicaciones únicamente llegamos a necesitar 1 o 2 componentes. Podemos deshabilitarlos usando el parámetro `exclude` de la función `load`:

In [ ]:
nlp = spacy.load("es_core_news_sm", exclude=["ner"])
print(nlp)

En este caso, cargamos un _pipeline_ con el componente `ner` deshabilitado, veamos:

In [ ]:
print(nlp.component_names)

['tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler', 'lemmatizer']


## **4. Objetos de Spacy**
---

En `spacy` generalmente estamos trabajando sobre tres tipos de objetos: los documentos `Doc`, las palabras `Token` y las secuencias `Span`.

Primero, vamos a definir un corpus con 3 documentos de prueba:

In [ ]:
texts = [
        "Alan Mathison Turing fue un matemático, lógico, informático teórico, criptógrafo, filósofo y biólogo teórico británico",
        "Marvin Lee Minsky fue un científico estadounidense. Es considerado uno de los padres de la inteligencia artificial. Fue cofundador del laboratorio de inteligencia artificial del Instituto de Tecnología de Massachusetts (MIT).",
        "Geoffrey Hinton es un informático británico. Hinton fue galardonado con el Premio Turing en 2018 junto con Yoshua Bengio y Yann LeCun por su trabajo en deep learning."
        ]

Ahora, podemos convertir cada documento a un `Doc` de `spacy` por medio del método `pipe` del *pipeline* como se muestra a continuación. El parámetro `n_process` nos permite ejecutar los componentes de forma paralelizada:

In [ ]:
corpus = list(nlp.pipe(texts, n_process=4))
print(corpus)

[Alan Mathison Turing fue un matemático, lógico, informático teórico, criptógrafo, filósofo y biólogo teórico británico, Marvin Lee Minsky fue un científico estadounidense. Es considerado uno de los padres de la inteligencia artificial. Fue cofundador del laboratorio de inteligencia artificial del Instituto de Tecnología de Massachusetts (MIT)., Geoffrey Hinton es un informático británico. Hinton fue galardonado con el Premio Turing en 2018 junto con Yoshua Bengio y Yann LeCun por su trabajo en deep learning.]


A simple vista, pareciera que los documentos no han cambiado mucho, no obstante, podemos validar que el tipo de un elemento dentro de `corpus` efectivamente es de tipo `Doc`:

In [ ]:
doc = corpus[0]
print(type(doc))

<class 'spacy.tokens.doc.Doc'>


Podemos extraer el texto asociado a un `Doc` con el atributo `text`:

In [ ]:
print(doc.text)

Alan Mathison Turing fue un matemático, lógico, informático teórico, criptógrafo, filósofo y biólogo teórico británico


El tipo `Doc` tiene distintos atributos que posteriormente usaremos para acceder a los resultados de los distintos componentes del modelo. Entre ellos, podemos obtener tokens a nivel de palabra con una simple indexación sobre el documento. Es decir, si queremos extraer la primera palabra del documento podemos hacer lo siguiente:

In [ ]:
token = doc[0]
print(token)

Alan


De nuevo pareciera que fuera un string, pero en realidad es un objeto de tipo `Token`:

In [ ]:
print(type(token))

<class 'spacy.tokens.token.Token'>


El objeto `Token` tiene distintos atributos asociados a cada componente como mostraremos posteriormente.

Por último, podemos extraer las oraciones del documento por medio del atributo `sents` para ver el tipo de datos que nos entrega:

In [ ]:
sents = list(doc.sents)
print(sents)

[Alan Mathison Turing fue un matemático, lógico, informático teórico, criptógrafo, filósofo y biólogo teórico británico]


Al igual que en los casos anteriores los resultados no son de tipo string sino que son de tipo `Span` (secuencia):

In [ ]:
sent = sents[0]
print(type(sent))

<class 'spacy.tokens.span.Span'>


El objeto `Span` también tiene distintos atributos asociados a cada componente como lo veremos más adelante. Cada elemento de un `Span` es un `Token`:

In [ ]:
token = sent[0]
print(type(token))

<class 'spacy.tokens.token.Token'>


## **5. Componentes**
---

Para ver el detalle de qué hace cada componente vamos a cargar el _pipeline_ `en_core_web_sm` en inglés, ya que, es la forma más estandarizada y general para entender los componentes y sus resultados.

En este caso no usamos el _pipeline_ en español ya que este no posee implementados todos los componentes que el _pipeline_ en inglés sí. Por ejemplo, no contiene etiquetas _POS_ de grano fino.

El ejemplo en español lo retomaremos más adelante, una vez veamos todos los componentes que puede llegar a tener un _pipeline_ de `spacy`.

Veamos cómo descargar el _pipeline_ `en_core_web_sm` para análisis del inglés:

In [ ]:
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


También lo cargamos:

In [ ]:
nlp = spacy.load("en_core_web_sm")

Veamos los componentes que trae este _pipeline_:

In [ ]:
print(nlp.component_names)

['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner']


Estos son los componentes típicos que trae un _pipeline_ de `spacy`. En algunos lenguajes puede cambiar `tagger` por `morphologizer` si no hay muchos modelos disponibles para etiquetado de palabras en un idioma en específico (`tagger` es más completo que `morphologizer`).

Vamos a ver el detalle de los componentes más importantes de `spacy` con el siguiente texto:

In [ ]:
text = """
Alan Mathison Turing was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist. Turing was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. He is widely considered to be the father of theoretical computer science and artificial intelligence.
"""

Ejecutamos el _pipeline_ para obtener un objeto de tipo `Doc`:

In [ ]:
doc = nlp(text.strip())

### **5.1. Tokenizer**
---
`spacy` trae un componente llamado `Tokenizer` que está definido por defecto en un paquete de lenguaje. Se trata de un componente que aplica reglas del lenguaje para separar palabras, detectar signos de puntuación y encontrar atributos generales de los objetos de tipo `Token`.

El `Tokenizer` nos permite obtener el listado de tokens (incluye signos de puntuación como tokens separados) por medio de algunas reglas específicas de cada lenguaje.

<img src="https://drive.google.com/uc?export=view&id=114bsjIq5vjLLLQYLwcUh14jObuUzbydy" width="60%">


Veamos cómo podemos obtener una lista de todas las palabras del documento con `spacy`:

In [ ]:
tokens = [token.text for token in doc]
print(tokens)

['Alan', 'Mathison', 'Turing', 'was', 'an', 'English', 'mathematician', ',', 'computer', 'scientist', ',', 'logician', ',', 'cryptanalyst', ',', 'philosopher', ',', 'and', 'theoretical', 'biologist', '.', 'Turing', 'was', 'highly', 'influential', 'in', 'the', 'development', 'of', 'theoretical', 'computer', 'science', ',', 'providing', 'a', 'formalisation', 'of', 'the', 'concepts', 'of', 'algorithm', 'and', 'computation', 'with', 'the', 'Turing', 'machine', ',', 'which', 'can', 'be', 'considered', 'a', 'model', 'of', 'a', 'general', '-', 'purpose', 'computer', '.', 'He', 'is', 'widely', 'considered', 'to', 'be', 'the', 'father', 'of', 'theoretical', 'computer', 'science', 'and', 'artificial', 'intelligence', '.']


Adicionalmente, desde el `Tokenizer` se calculan atributos en los tokens como:

* `is_stop`: si la palabra es un stopword, es decir, una palabra muy frecuente y poco informativa del lenguaje en cuestión (e.g., is, the, to, entre otras).
* `is_punct`: valida si el token es un signo de puntuación.
* `is_title`: valida si el token está en formato de título.
* `is_upper`: valida si el token está en mayúsculas.
* `is_lower`: valida si el token está en minúsculas.
* `lower_`: versión en minúsculas del token.

Por ejemplo, podemos validar qué tokens son _stopwords_:

In [ ]:
stops = [token.is_stop for token in doc]
print(stops)

[False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, True, True, False, True, False, False, False, False, False, True, False, True, True, False, True, False, True, False, True, True, False, False, False, True, True, True, False, True, False, True, True, False, False, False, False, False, True, True, False, False, True, True, True, False, True, False, False, False, True, False, False, False]


También podemos filtrar el listado de palabras que son _stopwords_:

In [ ]:
stops = list(filter(lambda token: not token.is_stop, doc))
print(stops)

[Alan, Mathison, Turing, English, mathematician, ,, computer, scientist, ,, logician, ,, cryptanalyst, ,, philosopher, ,, theoretical, biologist, ., Turing, highly, influential, development, theoretical, computer, science, ,, providing, formalisation, concepts, algorithm, computation, Turing, machine, ,, considered, model, general, -, purpose, computer, ., widely, considered, father, theoretical, computer, science, artificial, intelligence, .]


### **5.2. Senter**
---

El componente `senter` hace uso de la clase `Sentencizer` la cual permite segmentar el texto en distintas oraciones usando un modelo basado en reglas (expresiones regulares), el resultado queda almacenado en dos niveles:

* Como el atributo `is_sent_start` de cada `Token`.
* Como el atributo `sents` dentro del documento `Doc`.

<img src="https://drive.google.com/uc?export=view&id=1OtK3KZuKAeAwbAQpcmzGCz-CQxVfb192" width="80%">

Veamos qué tokens son inicio de oración dentro del documento que estamos manejando:

In [ ]:
sent_starts = list(filter(lambda token: token.is_sent_start, doc))
print(sent_starts)

[Alan, Turing, He]


También podemos extraer un listado de las oraciones del documento:

In [ ]:
sents = list(doc.sents)
print(sents)

[Alan Mathison Turing was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist., Turing was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer., He is widely considered to be the father of theoretical computer science and artificial intelligence.]


En total tenemos 3 oraciones:

In [ ]:
print(len(sents))

3


### **5.3. Tok2vec**
---

El componente `tok2vec` hace uso de la clase `Tok2Vec` para implementar distintos modelos de **embedding**, de los cuales hablaremos en detalle en la **Unidad 3**.

De forma resumida, este tipo de modelos permite extraer características numéricas a partir de palabras, oraciones o documentos y representarlas por medio de un vector. El vector se almacena como un atributo `vector` dentro de `Doc`, `Span` y `Token`. Esto es útil para capturar relaciones semánticas (relacionadas con el significado) en las palabras, oraciones o documentos. Como por ejemplo en el caso de la relación existente entre las palabras `Mujer` y `Hombre`, con respecto a las palabras `Reina` y `Rey`.

<img src="https://drive.google.com/uc?export=view&id=1w4z22p_X4KQ-sim0kZBnjkAZOaiojUCv" width="80%">

Veamos un ejemplo de cómo extraer características de todo un documento:

In [ ]:
vect = doc.vector
print(vect)

[ 1.73394039e-01 -2.96657056e-01  9.07820165e-02  2.51896027e-02
 -1.25062436e-01  1.52683094e-01  2.62303561e-01  3.90852354e-02
  1.67785317e-01 -8.08235332e-02  4.43081558e-02 -5.43820634e-02
 -1.94725946e-01  4.83091362e-02 -1.10306866e-01  8.31010640e-02
  3.40810418e-02  1.19768985e-01  2.94782058e-03 -8.48129690e-02
 -4.32738382e-03  2.18071550e-01 -1.12408072e-01 -2.42539912e-01
  2.06413463e-01  4.69311252e-02  1.66681662e-01 -2.30897531e-01
  5.49278140e-01  1.62225619e-01 -1.30502522e-01 -5.99215440e-02
  1.31596506e-01 -1.76367953e-01 -6.69770911e-02  8.53150785e-02
  1.82977822e-02  6.70244470e-02 -1.66366249e-02 -1.63158149e-01
 -5.23948409e-02  3.13756049e-01 -6.82974085e-02  1.19723804e-01
  2.25263327e-01  4.56906902e-03 -2.68668175e-01  8.93134922e-02
  2.26643026e-01 -8.98042247e-02 -1.35001212e-01  1.91183835e-01
 -1.68294802e-01 -1.54201105e-01 -9.46117342e-02 -5.62067777e-02
 -1.49681866e-01  2.42487147e-01  4.02724072e-02 -2.57585943e-02
 -1.30128726e-01 -8.57913

`spacy` extrae un vector de características como un arreglo de `numpy`:

In [ ]:
print(type(vect))

<class 'numpy.ndarray'>


También podemos extraer características de una oración:

In [ ]:
sent = list(doc.sents)[0]
print(type(sent))

<class 'spacy.tokens.span.Span'>


Veamos el vector:

In [ ]:
vect = sent.vector
print(vect)

[-2.99131647e-02 -4.48469013e-01  1.20129086e-01  4.80350181e-02
  6.46156899e-04  6.97114646e-01  2.27179959e-01  2.84380049e-01
  5.48247993e-01 -3.12370360e-01  4.07433093e-01 -3.07804674e-01
 -2.78902858e-01 -5.46625964e-02 -4.54434812e-01 -8.41911603e-03
  6.73473626e-02 -2.99910456e-02 -6.66361600e-02 -4.60449904e-01
 -4.90059108e-02  2.42209345e-01 -3.20486128e-02  4.84528877e-02
 -7.53362954e-04  2.43790075e-01  1.81988120e-01 -2.40989566e-01
  5.15884638e-01  4.01960105e-01 -3.17439698e-02 -2.12297440e-01
 -2.14072347e-01  6.83164224e-02  1.19584776e-01  1.98120072e-01
  1.51119545e-01  1.93580940e-01 -1.38739988e-01 -3.60125750e-01
 -2.24889979e-01  2.19920933e-01 -2.53928781e-01  2.10806414e-01
  2.03323588e-01  8.52216035e-02 -6.55967772e-01  7.39081502e-02
  2.50634402e-01 -1.52648911e-01  8.38948786e-02  1.67596430e-01
 -3.43088716e-01 -1.81491047e-01  3.06967776e-02 -1.48495868e-01
 -2.34027684e-01  5.52636623e-01  5.07424176e-02  1.84784800e-01
  7.40797725e-03 -5.21936

El proceso es similar para un `Token`:

In [ ]:
token = sent[0]
print(type(token))

<class 'spacy.tokens.token.Token'>


Veamos el vector:

In [ ]:
vect = token.vector
print(vect)

[-0.42797613 -0.9419694   1.9930038   0.12040219 -0.46856427  0.03224961
  1.4412253   1.1646957  -0.4544308   0.22364067  0.86232513 -0.62300736
  1.1953955  -0.8883365  -0.8108625  -0.55147576 -0.3091063   0.8109004
  0.8958614  -1.2029362  -1.0074904   0.4298711  -0.11933261 -0.29020375
  1.5298967   0.7843516  -0.65692663  0.08116288 -0.7826854   1.600491
 -0.7490417  -0.27093145 -0.5936148  -0.9173966  -1.0726113  -0.12619536
 -0.36907166  0.46580356 -0.69454885  1.7345712   0.13290189  0.6594495
 -1.3940576   1.0232966   0.073341    0.7874743  -1.1257935   0.18549141
  0.11692089 -0.19272856 -0.6598685   0.16561335 -0.2517925  -0.9145616
 -0.1824606  -0.7175338  -0.5754211  -0.24043652 -0.01884661  0.52194965
 -0.8727571   0.9630484  -0.18951893 -1.061382    0.28467822 -0.71741486
  0.42075944  3.3027873  -0.6467806  -1.0587168  -0.17276411  0.35101756
  0.7899697  -0.21453246 -0.16033967 -1.0446501  -0.20814872 -0.6535598
  0.538196   -0.21102162 -1.0957944  -0.7134103  -1.23517

En estos casos el vector tiene 96 características numéricas para representar el `Token`, `Sent` o el `Doc`:

In [ ]:
print(vect.shape)

(96,)


### **5.4. Tagger**
---

El componente `tagger` permite extraer etiquetas de tipo part-of-speech (POS) a cada `Token`. El proceso consiste en asignarle a cada palabra de una oración una etiqueta de acuerdo con su función gramatical dentro de la oración. Con POS podemos determinar si un token es un sustantivo, una preposición, un adjetivo, un adverbio, un verbo, complemento directo o indirecto, entre otros.

`spacy` maneja dos etiquetas POS dentro del texto:

* Para ver la etiqueta POS gruesa, use `token.pos_`.
* Para ver la etiqueta de grano fino, use `token.tag_`.

Para ver la descripción de cualquier tipo de etiqueta, use `spacy.explain(tag)`

#### **5.4.1. Etiquetas POS de Grano Grueso**
---

A cada token se le asigna una etiqueta POS de la siguiente lista si usamos el _pipeline_ estandarizado en inglés:

<table><tr><th>POS</th><th>Descripción</th><th>Ejemplos</th></tr>
    
<tr><td>ADJ</td><td>adjective</td><td>big, old, green, incomprehensible, first</td></tr>
<tr><td>ADP</td><td>adposition</td><td>in, to, during</td></tr>
<tr><td>ADV</td><td>adverb</td><td>very, tomorrow, down, where, there</td></tr>
<tr><td>AUX</td><td>auxiliary</td><td>is, has (done), will (do), should (do)</td></tr>
<tr><td>CONJ</td><td>conjunction</td><td>and, or, but</td></tr>
<tr><td>CCONJ</td><td>coordinating conjunction</td><td>and, or, but</td></tr>
<tr><td>DET</td><td>determiner</td><td>a, an, the</td></tr>
<tr><td>INTJ</td><td>interjection</td><td>psst, ouch, bravo, hello</td></tr>
<tr><td>NOUN</td><td>noun</td><td>girl, cat, tree, air, beauty</td></tr>
<tr><td>NUM</td><td>numeral</td><td>1, 2017, one, seventy-seven, IV, MMXIV</td></tr>
<tr><td>PART</td><td>particle</td><td>'s, not,</td></tr>
<tr><td>PRON</td><td>pronoun</td><td>I, you, he, she, myself, themselves, somebody</td></tr>
<tr><td>PROPN</td><td>proper noun</td><td>Mary, John, London, NATO, HBO</td></tr>
<tr><td>PUNCT</td><td>punctuation</td><td>., (, ), ?</td></tr>
<tr><td>SCONJ</td><td>subordinating conjunction</td><td>if, while, that</td></tr>
<tr><td>SYM</td><td>symbol</td><td>$, %, §, ©, +, −, ×, ÷, =, :), 😝</td></tr>
<tr><td>VERB</td><td>verb</td><td>run, runs, running, eat, ate, eating</td></tr>
<tr><td>X</td><td>other</td><td>sfpksdpsxmsa</td></tr>
<tr><td>SPACE</td><td>space</td></tr>
</table>

Veamos cómo extraer las etiquetas de grano grueso:

In [ ]:
pos_tags = [token.pos_ for token in doc]
print(pos_tags)

['PROPN', 'PROPN', 'PROPN', 'AUX', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'NOUN', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'CCONJ', 'ADJ', 'NOUN', 'PUNCT', 'VERB', 'AUX', 'ADV', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'PUNCT', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'CCONJ', 'NOUN', 'ADP', 'DET', 'VERB', 'NOUN', 'PUNCT', 'PRON', 'AUX', 'AUX', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'PUNCT', 'NOUN', 'NOUN', 'PUNCT', 'PRON', 'AUX', 'ADV', 'VERB', 'PART', 'AUX', 'DET', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'CCONJ', 'ADJ', 'NOUN', 'PUNCT']


Veamos una descripción más detallada de cada etiqueta usando el método `explain` y mostrando los datos como un dataframe:

In [ ]:
explained_tags = [spacy.explain(tag) for tag in pos_tags]
tokens = [token.text for token in doc]
pd.DataFrame({"token": tokens, "pos": pos_tags, "descripción": explained_tags})

,token,pos,descripción
0,Alan,PROPN,proper noun
1,Mathison,PROPN,proper noun
2,Turing,PROPN,proper noun
3,was,AUX,auxiliary
4,an,DET,determiner
...,...,...,...
72,science,NOUN,noun
73,and,CCONJ,coordinating conjunction
74,artificial,ADJ,adjective
75,intelligence,NOUN,noun


#### **5.4.1. Etiquetas POS de Grano Fino**
---

Las etiquetas de grano fino dan más detalles del tipo de palabra que estamos manejando, se pueden ver como subcategorías sobre las etiquetas de grano grueso (mayor detalle). Con el _pipeline_ general en inglés de `spacy` tenemos las siguientes etiquetas:

<table>
<tr><th>Grano Grueso</th><th>Grano Fino</th><th>Descripción</th><th>Ejemplo</th></tr>
<tr><td>ADJ</td><td>AFX</td><td>affix</td><td>The Flintstones were a **pre**-historic family.</td></tr>
<tr><td>ADJ</td><td>JJ</td><td>adjective</td><td>This is a **good** sentence.</td></tr>
<tr><td>ADJ</td><td>JJR</td><td>adjective, comparative</td><td>This is a **better** sentence.</td></tr>
<tr><td>ADJ</td><td>JJS</td><td>adjective, superlative</td><td>This is the **best** sentence.</td></tr>
<tr><td>ADJ</td><td>PDT</td><td>predeterminer</td><td>Waking up is **half** the battle.</td></tr>
<tr><td>ADJ</td><td>PRP\$</td><td>pronoun, possessive</td><td>**His** arm hurts.</td></tr>
<tr><td>ADJ</td><td>WDT</td><td>wh-determiner</td><td>It's blue, **which** is odd.</td></tr>
<tr><td>ADJ</td><td>WP\$</td><td>wh-pronoun, possessive</td><td>We don't know **whose** it is.</td></tr>
<tr><td>ADP</td><td>IN</td><td>conjunction, subordinating or preposition</td><td>It arrived **in** a box.</td></tr>
<tr><td>ADV</td><td>EX</td><td>existential there</td><td>**There** is cake.</td></tr>
<tr><td>ADV</td><td>RB</td><td>adverb</td><td>He ran **quickly**.</td></tr>
<tr><td>ADV</td><td>RBR</td><td>adverb, comparative</td><td>He ran **quicker**.</td></tr>
<tr><td>ADV</td><td>RBS</td><td>adverb, superlative</td><td>He ran **fastest**.</td></tr>
<tr><td>ADV</td><td>WRB</td><td>wh-adverb</td><td>**When** was that?</td></tr>
<tr><td>CONJ</td><td>CC</td><td>conjunction, coordinating</td><td>The balloon popped **and** everyone jumped.</td></tr>
<tr><td>DET</td><td>DT</td><td>determiner</td><td>**This** is **a** sentence.</td></tr>
<tr><td>INTJ</td><td>UH</td><td>interjection</td><td>**Um**, I don't know.</td></tr>
<tr><td>NOUN</td><td>NN</td><td>noun, singular or mass</td><td>This is a **sentence**.</td></tr>
<tr><td>NOUN</td><td>NNS</td><td>noun, plural</td><td>These are **words**.</td></tr>
<tr><td>NOUN</td><td>WP</td><td>wh-pronoun, personal</td><td>**Who** was that?</td></tr>
<tr><td>NUM</td><td>CD</td><td>cardinal number</td><td>I want **three** things.</td></tr>
<tr><td>PART</td><td>POS</td><td>possessive ending</td><td>Fred**'s** name is short.</td></tr>
<tr><td>PART</td><td>RP</td><td>adverb, particle</td><td>Put it **back**!</td></tr>
<tr><td>PART</td><td>TO</td><td>infinitival to</td><td>I want **to** go.</td></tr>
<tr><td>PRON</td><td>PRP</td><td>pronoun, personal</td><td>**I** want **you** to go.</td></tr>
<tr><td>PROPN</td><td>NNP</td><td>noun, proper singular</td><td>**Kilroy** was here.</td></tr>
<tr><td>PROPN</td><td>NNPS</td><td>noun, proper plural</td><td>The **Flintstones** were a pre-historic family.</td></tr>
<tr><td>VERB</td><td>MD</td><td>verb, modal auxiliary</td><td>This **could** work.</td></tr>
<tr><td>VERB</td><td>VB</td><td>verb, base form</td><td>I want to **go**.</td></tr>
<tr><td>VERB</td><td>VBD</td><td>verb, past tense</td><td>This **was** a sentence.</td></tr>
<tr><td>VERB</td><td>VBG</td><td>verb, gerund or present participle</td><td>I am **going**.</td></tr>
<tr><td>VERB</td><td>VBN</td><td>verb, past participle</td><td>The treasure was **lost**.</td></tr>
<tr><td>VERB</td><td>VBP</td><td>verb, non-3rd person singular present</td><td>I **want** to go.</td></tr>
<tr><td>VERB</td><td>VBZ</td><td>verb, 3rd person singular present</td><td>He **wants** to go.</td></tr>
</table>

Veamos cómo calcular las etiquetas de grano fino usando el atributo `tag_` del `Token`:

In [ ]:
tags = [token.tag_ for token in doc]
print(tags)

['NNP', 'NNP', 'NNP', 'VBD', 'DT', 'JJ', 'NN', ',', 'NN', 'NN', ',', 'NN', ',', 'NN', ',', 'NN', ',', 'CC', 'JJ', 'NN', '.', 'VBG', 'VBD', 'RB', 'JJ', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NN', 'NN', ',', 'VBG', 'DT', 'NN', 'IN', 'DT', 'NNS', 'IN', 'NNP', 'CC', 'NN', 'IN', 'DT', 'VBG', 'NN', ',', 'WDT', 'MD', 'VB', 'VBN', 'DT', 'NN', 'IN', 'DT', 'JJ', 'HYPH', 'NN', 'NN', '.', 'PRP', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NN', 'NN', 'CC', 'JJ', 'NN', '.']


Veamos una descripción más detallada de cada etiqueta usando el método `explain` y mostrando los datos como un dataframe:

In [ ]:
explained_tags = [spacy.explain(tag) for tag in tags]
tokens = [token.text for token in doc]
pd.DataFrame({"token": tokens, "tag": tags, "descripción": explained_tags})

,token,tag,descripción
0,Alan,NNP,"noun, proper singular"
1,Mathison,NNP,"noun, proper singular"
2,Turing,NNP,"noun, proper singular"
3,was,VBD,"verb, past tense"
4,an,DT,determiner
...,...,...,...
72,science,NN,"noun, singular or mass"
73,and,CC,"conjunction, coordinating"
74,artificial,JJ,"adjective (English), other noun-modifier (Chin..."
75,intelligence,NN,"noun, singular or mass"


Con las etiquetas de tipo POS podemos filtrar palabras de acuerdo a su función gramatical dentro del texto. Por ejemplo, podemos filtrar todos los nombres propios:

In [ ]:
propn = list(filter(lambda token: token.tag_ == "NNP", doc))
print(propn)

[Alan, Mathison, Turing, algorithm]


### **5.5. Parser**
---

El componente `parser` hace uso de la clase `DependencyParser` para estimar dependencias entre palabras de acuerdo a su función gramatical, en especial, se crean los siguientes atributos sobre un `Token`:

* `Token.dep_`: tipo de relación.
* `Token.head`: token con el que hay una relación.

El proceso de extracción de dependencias busca determinar relaciones y dependencias entre palabras en una oración con el fin de analizar su estructura gramatical.

<img src="https://drive.google.com/uc?export=view&id=1BgTWAed6WN57C-mQrN55-D8PWBDsVedJ" width="100%">

Existen distintos tipos de dependencias, las cuales puede encontrar en: [https://universaldependencies.org/u/dep/](https://universaldependencies.org/u/dep/). Por lo general, las etiquetas de dependencias se calculan a partir de las etiquetas POS y otros tipos de atributos. Veamos como listar las dependencias en `spacy` usando los atributos `dep_` y `head`:

In [ ]:
data = []
for token in doc:
    data.append(
            (token.text, token.dep_, token.head.text)
            )
pd.DataFrame(
        data,
        columns=["Token 1", "Relación", "Token 2"]
        )

,Token 1,Relación,Token 2
0,Alan,compound,Mathison
1,Mathison,compound,Turing
2,Turing,nsubj,was
3,was,ROOT,was
4,an,det,mathematician
...,...,...,...
72,science,pobj,of
73,and,cc,science
74,artificial,amod,intelligence
75,intelligence,conj,science


`spacy` cuenta con el módulo `displacy` para generar gráficas y entender las dependencias como un grafo.

Veamos un ejemplo donde usamos la función `render`, especificando el documento `doc`, el estilo del gráfico como árbol de dependencias `style = "dep"`, especificamos que el gráfico se mostrará en un notebook `jupyter=True` y algunas opciones que permiten modificar la forma en la que se muestra el diagrama:

In [ ]:
graph = spacy.displacy.render(
        doc,
        style="dep",
        jupyter=True,
        options={"distance": 110}
        )

Es posible configurar algunas opciones adicionales mediante el argumento  `options`:

<table>
<tr><th>Opción</th><th>Tipo</th><th>Descripción</th><th>Valor por defecto</th></tr>
<tr><td>`compact`</td><td>bool</td><td>"Modo compacto" con flechas cuadradas que ocupan menos espacio.</td><td>`False`</td></tr>
<tr><td>`color`</td><td>unicode</td><td>Color del texto (HEX, RGB o nombres de colores).</td><td>`#000000`</td></tr>
<tr><td>`bg`</td><td>unicode</td><td>Color del fondo (HEX, RGB o nombres de colores).</td><td>`#ffffff`</td></tr>
<tr><td>`font`</td><td>unicode</td><td>Fuente para todo el texto.</td><td>`Arial`</td></tr>
</table>

Para ver la lista de opciones completa: https://spacy.io/api/top-level#displacy_options

Veamos un ejemplo:

In [ ]:
options = {
        "distance": 110,
        "compact": "True",
        "color": "yellow",
        "bg": "#09a3d5",
        "font": "Times"
        }

graph = spacy.displacy.render(
        doc,
        style="dep",
        jupyter=True,
        options=options
        )

También, es posible exportar el gráfico como una imagen en formato `svg`:

In [ ]:
graph = spacy.displacy.render(
        doc,
        style="dep",
        jupyter=False,
        options=options
        )

Creamos el archivo:

In [ ]:
with open("deps.svg", "w") as f:
    f.write(graph)

### **5.6. Lemmatizer**
---

El componente `lemmatizer` se realiza por medio de la clase `Lemmatizer`, este proceso busca reducir una palabra a su raíz. Este procedimiento puede ser aplicado en aquellos casos donde no es necesario distinguir entre singulares y plurales de palabras o diversas conjugaciones de verbos regulares y deseamos agrupar estos tokens en una única representación, su raíz.

La _Lemmatization_ es uno de los procedimientos más sencillos para reducir una palabra a su raíz, consiste en cortar las palabras y agruparlas en una raíz común. Por ejemplo, _eat, eating y eaten_ son agrupadas en la palabra _eat_.

<img src="https://drive.google.com/uc?export=view&id=17D7d4bXcTuv4XQCT7-NrViKJgBF6hRED" width="75%">


El `Lemmatizer` agrega el atributo `lemma_` sobre cada token con su versión modificada.

Veamos cómo podemos extraer los lemmas de los tokens del documento que estamos manejando:

In [ ]:
lemmas = [token.lemma_ for token in doc]
print(lemmas)

['Alan', 'Mathison', 'Turing', 'be', 'an', 'english', 'mathematician', ',', 'computer', 'scientist', ',', 'logician', ',', 'cryptanalyst', ',', 'philosopher', ',', 'and', 'theoretical', 'biologist', '.', 'ture', 'be', 'highly', 'influential', 'in', 'the', 'development', 'of', 'theoretical', 'computer', 'science', ',', 'provide', 'a', 'formalisation', 'of', 'the', 'concept', 'of', 'algorithm', 'and', 'computation', 'with', 'the', 'ture', 'machine', ',', 'which', 'can', 'be', 'consider', 'a', 'model', 'of', 'a', 'general', '-', 'purpose', 'computer', '.', 'he', 'be', 'widely', 'consider', 'to', 'be', 'the', 'father', 'of', 'theoretical', 'computer', 'science', 'and', 'artificial', 'intelligence', '.']


A continuación podemos verificar solo aquellos tokens cuyo lemma sea diferente al texto original:

In [ ]:
lemmas = [(token.text, token.lemma_) for token in doc if token.text != token.lemma_]
print(lemmas)

[('was', 'be'), ('English', 'english'), ('Turing', 'ture'), ('was', 'be'), ('providing', 'provide'), ('concepts', 'concept'), ('Turing', 'ture'), ('considered', 'consider'), ('He', 'he'), ('is', 'be'), ('considered', 'consider')]


### **5.7. NER**
---

El componente `ner` hace uso de la clase `EntityRecognizer`, esta busca etiquetar segmentos del texto como entidades en un proceso conocido como reconocimiento de entidades nombradas (_Named Entity Recognition_ - NER).

NER es una tarea en procesamiento de lenguaje natural que sirve para extraer información estructurada a partir de un texto, por ejemplo: nombres de organizaciones, personas y lugares. También podemos usar NER para extraer entidades como nombres de productos, conceptos médicos, nombres de autores, nombres de marcas, entre otros.

Este componente asigna el atributo `Doc.ents` que nos permite identificar en qué lugar y de qué tipo es cada entidad como un objeto de tipo `Span`. Cada entidad tiene los siguientes atributos:

| Atributo | Descripción |
| --- | --- |
| `ent.text` | Texto de la entidad. |
| `ent.label_` | Descripción textual de la entidad. |
| `ent.start` | Índice que indica el comienzo de la entidad (tokens). |
| `ent.end` | Índice que indica el final de la entidad (tokens). |
| `ent.start_char` | Caracter inicial de la entidad (caracteres). |
| `ent.end_char` | Caracter final de la entidad (caracteres). |

Veamos un listado de las entidades en el documento que estamos usando de ejemplo:

In [ ]:
ents = doc.ents
print(ents)

(Alan Mathison Turing, English, Turing)


Podemos extraer información de las mismas y mostrarla como un dataframe:

In [ ]:
ent_text = [ent.text for ent in ents]
ent_type = [ent.label_ for ent in ents]
ent_start = [ent.start for ent in ents]
ent_end = [ent.end for ent in ents]
ent_startc = [ent.start_char for ent in ents]
ent_endc = [ent.end_char for ent in ents]
pd.DataFrame({
    "texto": ent_text, "ner": ent_type, "palabra_inicial": ent_start,
    "palabra_final": ent_end, "caracter_inicial": ent_startc,
    "caracter_final": ent_endc
    })

,texto,ner,palabra_inicial,palabra_final,caracter_inicial,caracter_final
0,Alan Mathison Turing,PERSON,0,3,0,20
1,English,LANGUAGE,5,6,28,35
2,Turing,ORG,45,46,297,303


El _pipeline_ general en inglés tiene las siguientes opciones para detección de entidades nombradas:

<table>
<tr><th>Tipo</th><th>Descripción</th><th>Ejemplo</th></tr>
<tr><td>PERSON</td><td>Personas, incluyendo personajes ficticios.</td><td>Fred Flintstone</td></tr>
<tr><td>NORP</td><td>Nacionalidades o grupos políticos o religiosos.</td><td>The Republican Party</td></tr>
<tr><td>FAC</td><td>Edificios, aeropuertos, autopistas, puentes, etc.</td><td>Logan International Airport, The Golden Gate</td></tr>
<tr><td>ORG</td><td>Compañías, agencias, instituciones, etc.</td><td>Microsoft, FBI, MIT</td></tr>
<tr><td>GPE</td><td>Países, ciudades, estados.</td><td>France, UAR, Chicago, Idaho</td></tr>
<tr><td>LOC</td><td>Ubicaciones no geopolíticas, cadenas montañosas, cuerpos de agua.</td><td>Europe, Nile River, Midwest</td></tr>
<tr><td>PRODUCT</td><td>Objetos, vehículos, comidas, etc. (No servicios.)</td><td>Formula 1</td></tr>
<tr><td>EVENT</td><td>Huracanes nombrados, batallas, guerras, eventos deportivos, etc.</td><td>Olympic Games</td></tr>
<tr><td>WORK_OF_ART</td><td>Títulos de libros, canciones, pinturas, etc.</td><td>The Mona Lisa</td></tr>
<tr><td>LAW</td><td>Documentos nombrados convertidos en leyes.</td><td>Roe v. Wade</td></tr>
<tr><td>LANGUAGE</td><td>Cualquier idioma.</td><td>English</td></tr>
<tr><td>DATE</td><td>Fechas o periodos absolutos o relativos.</td><td>*20 July 1969*</td></tr>
<tr><td>TIME</td><td>Periodos de tiempo inferiores a un día.</td><td>Four hours</td></tr>
<tr><td>PERCENT</td><td>Porcentaje, incluyendo "%".</td><td>Eighty percent</td></tr>
<tr><td>MONEY</td><td>Valores monetarios, incluyendo unidades.</td><td>Twenty Cents</td></tr>
<tr><td>QUANTITY</td><td>Medidas, como de peso o distancia.</td><td>Several kilometers, 55kg</td></tr>
<tr><td>ORDINAL</td><td>Valores ordinales (primero, segundo, tercero, etc.).</td><td>9th, Ninth</td></tr>
<tr><td>CARDINAL</td><td>Numerales que no entran en otra categoría.</td><td>2, Two, Fifty-two</td></tr>
</table>

Adicional a esto, podemos usar `displacy` para visualizar entidades nombradas del texto. La única diferencia con respecto a la visualización de dependencias es que en este caso pasamos como argumento `"ent"` al parámetro `style`, de la siguiente forma:

In [ ]:
graph = spacy.displacy.render(
        doc,
        style="ent",
        jupyter=True,
        options={"distance": 110}
        )